In [55]:
import MetaTrader5 as mt5
import pandas as pd
import datetime
import sys
import os
import mplfinance as mpf
import talib as ta
import numpy as np
import pandas_ta as ta

import warnings
warnings.filterwarnings('ignore')

from backtesting import Strategy, Backtest
from backtesting.test import SMA, GOOG
from backtesting.lib import crossover, TrailingStrategy

current = os.path.abspath('')
parent = os.path.dirname(current)
sys.path.append(parent)

from trade_utils import get_symbol_data, get_MT5_Timeframe

if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
    
symbol = "ETHUSD"

data = get_symbol_data(symbol, closed_candles_only=True, periods=90000, timeframe="5M")

data.rename(columns={"open": "Open", "close": "Close", "low": "Low", "high": "High", "volume": "Volume"}, inplace=True)

data.sort_index(ascending=True, inplace=True)

# drop all na's from our indicators, patterns and signal columns
data.dropna(subset=["signal_rsi_sell", "signal_rsi_buy", "ind_atr","ptn_bullish_engulfing_1", "ptn_bearish_engulfing_1"], inplace=True)

get symbol history


In [122]:


class MyStrategy(Strategy):
    # 5M = 16
    # 15M = 16
    # 30M = 4 ( too long)
    # 1H = 5 ( too long too much drawdown )
    upper_bound = 75
    lower_bound = 25
    rsi_window = 16
    
    #sl_amount = 5
    atr_multiplier = 0.5    #5.5 
    
    def init(self): 
        # before strategy is run
        # calculate indicators and other things used by strategy
        super().init()

        self.rsi = self.I(ta.rsi, pd.Series(self.data.Close), self.rsi_window)
        
    def next(self):
        # each candlestick
        super().next()


        style = "third"
        
        if style == "first":
            # buy only
            if crossover(self.rsi, self.upper_bound):
                self.position.close()
            elif crossover(self.lower_bound, self.rsi):
                stop_loss = self.data.Close[-1] - ( self.data.ind_atr[-1] * self.atr_multiplier)
                self.buy(sl=stop_loss)
                self.buy()
        if style == "second":
            # sell only
            if crossover(self.rsi, self.lower_bound):
                self.position.close()
            elif crossover(self.upper_bound, self.rsi):
                stop_loss = self.data.Close[-1] - ( self.data.ind_atr[-1] * self.atr_multiplier)
                self.buy(sl=stop_loss)
                self.buy()
        
        elif style == "third": 
            # if a positionis currentyl open the check to see if we need to close
            if self.position:
                if self.position.is_long:
                    if crossover(self.rsi, self.upper_bound):
                        self.position.close()
                else:
                    if crossover(self.rsi, self.lower_bound):
                        self.position.close()
                            
            # if no position is open
            if not self.position:
                if crossover(self.rsi, self.upper_bound) and self.rsi > self.upper_bound:
                    # over bought
                    stop_loss = self.data.Close[-1] + ( self.data.ind_atr[-1] * self.atr_multiplier)

                    self.sell()
                elif crossover(self.rsi, self.lower_bound) and self.rsi < self.lower_bound:
                    # over sold
                    stop_loss = self.data.Close[-1] - ( self.data.ind_atr[-1] * self.atr_multiplier)

                    self.buy()


bt = Backtest(data, MyStrategy, cash=50_000)
stats = bt.run()
print(stats)

Start                     2023-04-15 04:00:00
End                       2024-02-28 16:45:00
Duration                    319 days 12:45:00
Exposure Time [%]                   46.657776
Equity Final [$]                     40687.98
Equity Peak [$]                      59408.78
Return [%]                          -18.62404
Buy & Hold Return [%]               58.218217
Return (Ann.) [%]                  -20.948586
Volatility (Ann.) [%]               26.739202
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -35.1815
Avg. Drawdown [%]                   -1.059246
Max. Drawdown Duration      189 days 07:35:00
Avg. Drawdown Duration        3 days 22:12:00
# Trades                                  121
Win Rate [%]                        62.809917
Best Trade [%]                       5.862239
Worst Trade [%]                    -10.056736
Avg. Trade [%]                    

In [111]:
stats._trades

stats._trades.sort_values(by="Duration", ascending=False)

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
39,26,15385,16617,1754.62,1705.456901,-1278.240566,-0.028019,2023-06-10 02:10:00,2023-06-14 16:05:00,4 days 13:55:00
175,27,76509,77653,2502.98,2605.040000,2755.620000,0.040775,2024-01-12 16:45:00,2024-01-16 16:35:00,3 days 23:50:00
95,30,38685,39814,1608.73,1643.190000,1033.800000,0.021421,2023-09-01 13:15:00,2023-09-05 11:20:00,3 days 22:05:00
91,30,36149,37103,1659.05,1651.830000,-216.600000,-0.004352,2023-08-23 17:55:00,2023-08-27 01:25:00,3 days 07:30:00
77,27,30413,31317,1830.97,1837.200000,168.210000,0.003403,2023-08-03 18:10:00,2023-08-06 21:30:00,3 days 03:20:00
...,...,...,...,...,...,...,...,...,...,...
86,27,34365,34367,1764.20,1733.818188,-820.308921,-0.017221,2023-08-17 11:30:00,2023-08-17 11:40:00,0 days 00:10:00
96,30,40614,40615,1633.32,1621.720275,-347.991753,-0.007102,2023-09-08 06:00:00,2023-09-08 06:05:00,0 days 00:05:00
66,25,25556,25557,1911.53,1896.332828,-379.929311,-0.007950,2023-07-17 06:20:00,2023-07-17 06:25:00,0 days 00:05:00
2,24,1068,1069,2028.38,1979.694989,-1168.440276,-0.024002,2023-04-19 04:15:00,2023-04-19 04:20:00,0 days 00:05:00


In [ ]:
stats = bt.optimize(
        #sl_amount = range(5, 100, 5),
        atr_multiplier = [.5, .75, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5],
        #upper_bound = range(60,85,2),
        #lower_bound = range(20,40,2),
        rsi_window = range(4,20,1),
        #maximize='Equity Final [$]'
)


print(stats["_strategy"].upper_bound, 
      stats["_strategy"].lower_bound, 
      stats["_strategy"].rsi_window, 
      stats["_strategy"].atr_multiplier)

#print(stats["_strategy"].upper_bound, stats["_strategy"].lower_bound)
#print(stats["_strategy"].rsi_window)
#print(stats["_strategy"].atr_multiplier)

